In [1]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import font
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 17,
        "RightPoke": 27,
        "Pellet": 22,
    },
    'Port 2': {
        "LeftPoke": 10,
        "RightPoke": 9,
        "Pellet": 11,
    },
    'Port 3': {
        "LeftPoke": 0,
        "RightPoke": 5,
        "Pellet": 6,
    },
    'Port 4': {
        "LeftPoke": 13,
        "RightPoke": 19,
        "Pellet": 26,
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke event triggered.")
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke with pellet, signal triggered.")
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke with pellet, signal triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

# Splash screen class
class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)

        screen_width = 800
        screen_height = 480
        width, height = 600, 200
        x, y = (screen_width - width) // 2, (screen_height - height) // 2
        self.root.geometry(f"{width}x{height}+{x}+{y}")
        self.root.configure(bg="black")

        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Helvetica", 24, "bold"), bg="black", fg="orange")
        self.label.pack(expand=True)
        self.root.after(duration, self.close_splash)

    def close_splash(self):
        self.root.destroy()

# Main GUI Application Class
class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED DATA MONITOR V.01")
        self.root.geometry("800x480")

        # Configure main frame
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))

        # Port widgets, queues, and controls
        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}

        # Add port display and controls
        self.setup_ports()
        self.create_controls()
        self.root.after(100, self.update_gui)

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name, padding="3")
            frame.grid(column=idx % 2, row=idx // 2, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 10), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=28, height=6, wrap=tk.WORD)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {'frame': frame, 'status_label': status_label, 'text_widget': text_widget}
            self.port_queues[port_name] = queue.Queue()

    def create_controls(self):
        tk.Label(self.mainframe, text="Experimenter:", font=("Helvetica", 8)).grid(column=0, row=3, sticky=tk.W)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name, width=12)
        self.experimenter_entry.grid(column=1, row=3, sticky=tk.W)

        tk.Label(self.mainframe, text="Experiment Name:", font=("Helvetica", 8)).grid(column=2, row=3, sticky=tk.W)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name, width=12)
        self.experiment_entry.grid(column=3, row=3, sticky=tk.W)

        self.start_button = tk.Button(self.mainframe, text="START", font=("Helvetica", 10, "bold"), bg="green", command=self.start_experiment)
        self.start_button.grid(column=0, row=4, padx=5, pady=5)

        self.stop_button = tk.Button(self.mainframe, text="STOP", font=("Helvetica", 10, "bold"), bg="red", command=self.stop_experiment)
        self.stop_button.grid(column=1, row=4, padx=5, pady=5)

        self.browse_button = tk.Button(self.mainframe, text="Browse Data Folder", font=("Helvetica", 10), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=2, row=4, padx=5, pady=5, columnspan=2, sticky=tk.E)

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def start_experiment(self):
        # Placeholder for start experiment logic
        pass

    def stop_experiment(self):
        stop_event.set()
        for t in self.threads:
            t.join()
        GPIO.cleanup()
        self.save_all_data()
        self.root.quit()

    def save_all_data(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        for port_identifier, data_rows in self.data_to_save.items():
            filename_user = f"{self.experiment_folder}/{port_identifier}.csv"
            with open(filename_user, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                writer.writerows(data_rows)

# Main execution
if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_7183/2132770344.py", line 221, in stop_experiment
    for t in self.threads:
             ^^^^^^^^^^^^
AttributeError: 'FED3MonitorApp' object has no attribute 'threads'
